Code for peak shaving optimization

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import gurobipy

In [3]:
demand = pd.read_csv('./data/ColumbiaDemand.csv')
demand.head()
#Restructure into one list of demands for each month

,Date,TotalDemand [kWh],Period01 [kWh],Period02 [kWh],Period03 [kWh],Period04 [kWh],Period05 [kWh],Period06 [kWh],Period07 [kWh],Period08 [kWh],...,Period87 [kWh],Period88 [kWh],Period89 [kWh],Period90 [kWh],Period91 [kWh],Period92 [kWh],Period93 [kWh],Period94 [kWh],Period95 [kWh],Period96 [kWh]
0,01-Jan-2018,72686.88,734.40,731.52,727.20,735.84,730.08,728.64,731.52,730.08,...,740.16,741.60,734.40,735.84,735.84,730.08,725.76,731.52,728.64,724.32
1,02-Jan-2018,73594.08,728.64,727.20,728.64,727.20,732.96,734.40,734.40,737.28,...,743.04,741.60,741.60,738.72,750.24,743.04,750.24,738.72,738.72,743.04
2,03-Jan-2018,73440.00,745.92,743.04,745.92,744.48,741.60,734.40,735.84,734.40,...,734.40,732.96,721.44,714.24,721.44,725.76,727.20,724.32,731.52,722.88
3,04-Jan-2018,73967.04,718.56,718.56,724.32,727.20,720.00,724.32,725.76,720.00,...,747.36,744.48,735.84,734.40,741.60,743.04,738.72,741.60,740.16,735.84
4,05-Jan-2018,74625.12,745.92,743.04,744.48,741.60,737.28,743.04,747.36,748.80,...,767.52,768.96,760.32,758.88,756.00,756.00,753.12,754.56,750.24,745.92


In [4]:
#Define variables
days = 31 #28 or 30 depending on the month
periods = 96
# Battery Efficiency
eta = np.array([.95, .7]) #.95 or .7
# Power Rating
P = np.array([300, 500]) # 300 or 500
# Energy Rating
E = np.array([200, 50]) # 200 or 50
# Demand
# D = january_list, feb_list, ...
# Peak Demand Charge
# On peak month: June, July, August, September
#       8am-6pm, 8am-10pm, all-day
# Off peak month: other
#       8am-10pm, all-day
C = np.array([[9.15, 18.44, 16.66],
            [13.96, 4.21]])
# B is some selection of C and its summation
#Battery duration E*P -> if P=1, then E=4 or E=12, if P=2, then E=8 or E=24
bat_dur = np.array([4, 12])

# Decision Variables
#Storage discharge power
d = cp.Variable((days*periods), nonneg=True)
# Storage charge power
q = cp.Variable((days*periods), nonneg=True)
# Energy storged
e = cp.Variable((days*periods), nonneg=True)
# peak demand
p = cp.Variable(nonneg=True)


/var/folders/8w/yzz7q0p146x4qrt7dwp0mhy40000gn/T/ipykernel_69682/3273664124.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  C = np.array([[9.15, 18.44, 16.66],


In [ ]:
#Define Objective - BP + sum over t
obj = cp.Minimize(B*p + C*sum(D - d - q))